## Linking without deduplication

A simple record linkage model using the `link_only` [link type](https://moj-analytical-services.github.io/splink/settings_dict_guide.html#link_type).


In [1]:
from splink.datasets import splink_datasets

df = splink_datasets.fake_1000

# Split a simple dataset into two, separate datasets which can be linked together.
df_l = df.sample(frac=0.5)
df_r = df.drop(df_l.index)

df_l.head(2)

,unique_id,first_name,surname,dob,city,email,cluster
28,28,Thomas,Gabriel,1976-09-15,London,gabriel.t54@nichols.info,11
245,245,Joshua,NaN,2002-08-08,Portsmouth,j.williams@levine-johnson.com,64


In [8]:
from splink.linker import Linker
from splink.database_api import DuckDBAPI
from splink.blocking_rule_library import block_on
import splink.comparison_library as cl
import splink.comparison_template_library as ctl


settings = {
    "link_type": "link_only",
    "blocking_rules_to_generate_predictions": [
        block_on("first_name"),
        block_on("surname"),
    ],
    "comparisons": [
        ctl.NameComparison(
            "first_name",
        ),
        ctl.NameComparison("surname"),
        # TODO: Fix date comparison
        # ctl.date_comparison("date_of_birth",
        #                     damerau_levenshtein_thresholds=[],
        #                     cast_strings_to_date=True,
        #                     invalid_dates_as_null=True,
        #                     date_format="%Y%m%d"),
        cl.LevenshteinAtThresholds("dob", [2]),
        cl.ExactMatch("city").configure(term_frequency_adjustments=True),
        ctl.EmailComparison("email", include_username_fuzzy_level=False),
    ],
}

linker = Linker(
    [df_l, df_r],
    settings,
    database_api=DuckDBAPI(),
    input_table_aliases=["df_left", "df_right"],
)

In [9]:
linker.completeness_chart(cols=["first_name", "surname", "dob", "city", "email"])

alt.LayerChart(...)

In [13]:
from splink.blocking_rule_library import block_on, BlockingRuleCreator

deterministic_rules = [
    "l.first_name = r.first_name and levenshtein(r.dob, l.dob) <= 1",
    "l.surname = r.surname and levenshtein(r.dob, l.dob) <= 1",
    "l.first_name = r.first_name and levenshtein(r.surname, l.surname) <= 2",
    block_on("email"),
]

# TODO: estimate_probability_two_random_records_match should accept
# BlockingRuleCreator objects
deterministic_rules = [
    br.get_blocking_rule("duckdb") if isinstance(br, BlockingRuleCreator) else br
    for br in deterministic_rules
]

linker.estimate_probability_two_random_records_match(deterministic_rules, recall=0.7)

Probability two random records match is estimated to be  0.00331.
This means that amongst all possible pairwise record comparisons, one in 301.72 are expected to match.  With 250,000 total possible comparisons, we expect a total of around 828.57 matching pairs


In [14]:
linker.estimate_u_using_random_sampling(max_pairs=1e6, seed=1)

----- Estimating u probabilities using random sampling -----

Estimated u probabilities using random sampling

Your model is not yet fully trained. Missing estimates for:
    - first_name (no m values are trained).
    - surname (no m values are trained).
    - dob (no m values are trained).
    - city (no m values are trained).
    - email (no m values are trained).


In [15]:
session_dob = linker.estimate_parameters_using_expectation_maximisation(block_on("dob"))
session_email = linker.estimate_parameters_using_expectation_maximisation(
    block_on("email")
)
session_first_name = linker.estimate_parameters_using_expectation_maximisation(
    block_on("first_name")
)


----- Starting EM training session -----



Estimating the m probabilities of the model by blocking on:
l."dob" = r."dob"

Parameter estimates will be made for the following comparison(s):
    - first_name
    - surname
    - city
    - email

Parameter estimates cannot be made for the following comparison(s) since they are used in the blocking rules: 
    - dob

Iteration 1: Largest change in params was -0.391 in the m_probability of email, level `Exact match on email`
Iteration 2: Largest change in params was 0.121 in probability_two_random_records_match
Iteration 3: Largest change in params was 0.0381 in probability_two_random_records_match
Iteration 4: Largest change in params was 0.0143 in probability_two_random_records_match
Iteration 5: Largest change in params was 0.00639 in probability_two_random_records_match
Iteration 6: Largest change in params was 0.00319 in probability_two_random_records_match
Iteration 7: Largest change in params was 0.00183 in probability_two_random_records_match
Iteration 8: Largest change in pa

In [16]:
results = linker.predict(threshold_match_probability=0.9)

In [17]:
results.as_pandas_dataframe(limit=5)

,match_weight,match_probability,source_dataset_l,source_dataset_r,unique_id_l,unique_id_r,first_name_l,first_name_r,gamma_first_name,surname_l,...,dob_l,dob_r,gamma_dob,city_l,city_r,gamma_city,email_l,email_r,gamma_email,match_key
0,3.312734,0.908563,df_left,df_right,589,588,None,Jacob,-1,Riley,...,2012-09-30,2012-10-01,0,Livopeol,Liverpool,0,j.riley9@bennett.net,j.riley9tbennett.ne@,1,1
1,3.370957,0.911861,df_left,df_right,808,811,Elliott,Elliott,3,Byrne,...,2002-01-31,2002-03-04,0,Stokeno--Trent,Stoke-on-Trent,0,e.b30@little.biz,e.b30@little.biz,3,0
2,3.370957,0.911861,df_left,df_right,317,315,Daisy,Daisy,3,None,...,1991-04-16,2001-05-11,0,Manshecter,Manchester,0,dh@powell.net,dh@powell.net,3,0
3,3.370957,0.911861,df_left,df_right,448,450,Aisha,Aisha,3,BryBant,...,1993-08-08,1993-07-10,0,Bradfrod,Brddfora,0,aishab64@obrien-flores.com,aishab64@obrien-flores.com,3,0
4,3.370957,0.911861,df_left,df_right,790,786,Jackson,Jackson,3,Fisreh,...,2004-12-26,2013-12-25,0,Lodon,London,0,j.fisher4@sullivan.com,j.fisher4@sullivan.com,3,0
